In [1]:
import pandas as pd
import numpy as np
import random

#normalizing each attribute in the training set so no one attribute dominates the classification
def normalize(data):
    normdata=pd.DataFrame(index=data.index.tolist(),columns = data.columns.tolist())
    for (columname, columnData) in data.iteritems():
        if columname!='clss':
            normdata[columname] = [(data.loc[index,columname]-data[columname].min())/
                                   (data[columname].max()-data[columname].min()) for index,row in data.iterrows()]
        else:
            normdata[columname]=data[columname]
    
    return normdata

#euclidean distance between each example in the training set and the new example we want to classify
def distance(tset,v):
    tset['dist']=0
    for index,row in tset.iterrows():
        tset.loc[index,'dist']=np.linalg.norm(tset.iloc[index,:-2].to_numpy()-v.iloc[0].to_numpy())
    return tset

#classifies the new point by choosing the class that comes up the most times in the k smallest distances  
def classify(tset,k,v):
    normtset=normalize(tset)
    normtset = distance(normtset,v)
    nn=normtset.nsmallest(k,'dist')
    clss = pd.DataFrame(nn['clss'].value_counts())
    
    return "most probable class is "+ str(clss.index[0])
        

In [2]:
#example training set from chapter 1 (or 2 i forgot)
z=pd.read_csv("example1.csv",header=0)
z=z.rename(columns ={'class':'clss'})
v = pd.DataFrame({'at1': [3.1], 'at2': [2], 'at3': [1.99]})

classify(z,3,v)

'most probable class is pos'

In [28]:
#training set from UCI repository https://archive.ics.uci.edu/ml/datasets/Wine
wine_data = pd.read_csv('wines.csv')
wine_data= wine_data.rename(columns ={'class':'clss'})
#next two lines are me moving the class of each example to the end of the df bc its easier
#theres obviously a way to classify/get distance without changing the df but o well i already did it this way


temp=wine_data.pop('clss')
wine_data['clss']=temp

#normalizing our training set
normalized_wine_data=normalize(wine_data)

#creating an example we want to classify
test_set=pd.DataFrame(index=[0],columns=normalized_wine_data.columns.tolist())
test_set=test_set.fillna(0)

#populating our example with random numbers that fall in the range of already existing data 
for (columname,columnData) in normalized_wine_data.iteritems():
    test_set[columname]=round(random.uniform(normalized_wine_data[columname].min(),normalized_wine_data[columname].max()),5)
test_set=test_set.drop(columns='clss')

#see what happens for different values of k
results={}
for k in range(1,15,2):
    results[f'{k}NN']=(classify(normalized_wine_data,k,test_set))

    
print(results)

{'1NN': 'most probable class is 3', '3NN': 'most probable class is 3', '5NN': 'most probable class is 1', '7NN': 'most probable class is 1', '9NN': 'most probable class is 1', '11NN': 'most probable class is 1', '13NN': 'most probable class is 1'}
